In [119]:
import glob
import random
import os
import numpy as np
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms
import numpy as np
from torch.utils.data import DataLoader
from models import *
from datasets import *
files = sorted(glob.glob(os.path.join("/media/arg_ws3/5E703E3A703E18EB/data/MM_BOX/test/A") + '/*.*'))
"depth_" + files[1].split("/")[-1]

'depth_1006_original.png'

In [98]:
counter = 0
for dirPath, dirNames, fileNames in os.walk("/media/arg_ws3/5E703E3A703E18EB/data/subt_real/depth/extinguisher/"):
    #print(dirPath)
    for f in fileNames:
        counter = counter + 1
        print( os.path.join(dirPath, f))
print(counter)
name_A = os.path.join(dirPath, f)

/media/arg_ws3/5E703E3A703E18EB/data/subt_real/depth/extinguisher/scene000001/0.png
/media/arg_ws3/5E703E3A703E18EB/data/subt_real/depth/extinguisher/scene000001/1.png
/media/arg_ws3/5E703E3A703E18EB/data/subt_real/depth/extinguisher/scene000001/10.png
/media/arg_ws3/5E703E3A703E18EB/data/subt_real/depth/extinguisher/scene000001/11.png
/media/arg_ws3/5E703E3A703E18EB/data/subt_real/depth/extinguisher/scene000001/12.png
/media/arg_ws3/5E703E3A703E18EB/data/subt_real/depth/extinguisher/scene000001/13.png
/media/arg_ws3/5E703E3A703E18EB/data/subt_real/depth/extinguisher/scene000001/14.png
/media/arg_ws3/5E703E3A703E18EB/data/subt_real/depth/extinguisher/scene000001/15.png
/media/arg_ws3/5E703E3A703E18EB/data/subt_real/depth/extinguisher/scene000001/16.png
/media/arg_ws3/5E703E3A703E18EB/data/subt_real/depth/extinguisher/scene000001/17.png
/media/arg_ws3/5E703E3A703E18EB/data/subt_real/depth/extinguisher/scene000001/18.png
/media/arg_ws3/5E703E3A703E18EB/data/subt_real/depth/extinguisher/s

In [115]:
split_name = name_A.replace('depth', '.').split('.')[1]
B_name = "/media/arg_ws3/5E703E3A703E18EB/data/subt_real/" + "image" + split_name + '.jpg'
img_B = Image.open(B_name)
img_B.save('ddd.jpg')

In [ ]:
/media/arg_ws3/5E703E3A703E18EB/data/subt_real/image/extinguisher/scene000018/13.jpg
/media/arg_ws3/5E703E3A703E18EB/data/subt_real/image/extinguisher/scene000018/9.png

In [26]:
img_A = Image.open('/media/arg_ws3/5E703E3A703E18EB/data/subt_real/depth/extinguisher/scene000001/7.png')
#img_A = Image.fromarray(np.array(img_A)[:, ::-1, :], 'RGB')
print(img_A.size)
img_A.save('out1.png')
print(img_A.mode)
#img_A = img_A/1000.
print(type(img_A))
img_A = np.array(img_A)

#img_A = np.asarray(img_A, dtype=np.uint8)
print(type(img_A))

(640, 480)
I
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'numpy.ndarray'>


In [43]:
print(img_A[255,400], img_A[255,400])
img_A = img_A
print(img_A[255,400], img_A[255,400])
img_A = Image.fromarray(img_A)
print(type(img_A))
print(img_A.mode)
img_A = img_A.convert('RGB')
print(img_A.mode)
print(img_A.getpixel((255,256)))
img_A.save('out2.png')

2063 2063
2063 2063
<class 'PIL.Image.Image'>
I
RGB
(255, 255, 255)


In [1]:
import glob
import random
import os
import numpy as np
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms
import numpy as np
from torch.utils.data import DataLoader
from models import *
from datasets import *

In [2]:
transforms_ = [ transforms.Resize((480, 640), Image.BICUBIC),
                transforms.ToTensor(),
                transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)) ]
dataloader = DataLoader(ImageDataset("/media/arg_ws3/5E703E3A703E18EB/data/subt_real/", transforms_=transforms_),
                        batch_size=1, shuffle=True, num_workers=4)

282


In [3]:
batch_iterator = iter(dataloader)
images = next(batch_iterator)

In [49]:
a = Image.open('sss.jpg')
print(a.getpixel((0, 55)))
b = Image.open('ss.jpg')
print(b.getpixel((0, 55)))

(179, 184, 188)
(181, 185, 186)
